In [94]:
import pandas as pd
import numpy as np
import math 

select sample stock

In [95]:
ticker_name = "AMZN"

In [96]:
SPY = pd.read_csv("SPY.csv")
ticker = pd.read_csv(ticker_name+".csv")

In [97]:
ticker_date = [x for x in ticker.Date]
ticker_ = SPY[SPY.Date >= ticker_date[0]]

In [98]:
ticker_ = ticker_[["Date","Close","Volume"]]

In [99]:
ticker = ticker[["Date","Close","Volume"]]

In [100]:
ticker = pd.merge(ticker, ticker_, on = "Date", how='left')

In [101]:
ticker.columns = ["Date", ticker_name+"_close" , ticker_name+"_volume" ,"SPY_close","SPY_volume"]

In [87]:
def cal_corr(time_window = 30):
    ticker_corr = ticker[[ticker_name+"_close","SPY_close"]].rolling(time_window).corr()
    ticker_corr = ticker_corr[ticker_name+"_close"]
    ticker_corr.index = range(len(ticker_corr))
    ticker_corr = ticker_corr[ticker_corr.index %2 == 1 ]
    ticker_corr.index = range(len(ticker_corr))
    return ticker_corr

def cal_vol(time_window = 30):
    ticker_std = ticker[[ticker_name+"_close"]].rolling(time_window).std()
    return ticker_std

def cal_rtn(time_window = 30):
    return ticker[ticker_name+"_close"].apply(lambda x: math.log(x)).diff().fillna(0).rolling(time_window).sum().fillna(0)

def cal_volume(time_window = 30):
    return ticker[ticker_name+"_volume"].rolling(time_window).sum().fillna(0)


    

# add function 

In [88]:
def add_feature(dataframe):
    dataframe["value_to_predict"] = cal_corr(44).shift(-43)
    window = [3,6,10,15,20,30,60,100,150]
    for i in window:
        dataframe["corr_"+str(i)] = cal_corr(i)
        
    for i in window:
        dataframe["vol_"+str(i)] = cal_vol(i)
        
    for i in window:
        dataframe["rtn_"+str(i)] = cal_rtn(i)
    
    for i in window:
        dataframe["volume_"+str(i)] = cal_volume(i)
        
    return dataframe
    
 
    

    
    
    

In [89]:
ticker = add_feature(ticker)

In [90]:
ticker = ticker.dropna()

In [91]:
ticker

,Date,JPM_close,JPM_volume,SPY_close,SPY_volume,value_to_predict,corr_3,corr_6,corr_10,corr_15,...,rtn_150,volume_3,volume_6,volume_10,volume_15,volume_20,volume_30,volume_60,volume_100,volume_150
149,2000-08-04,52.437500,8546500,146.375000,3686600,0.844337,0.392518,0.775665,0.565606,0.532364,...,0.076341,21153600.0,38241900.0,52381700.0,76599400.0,98287100.0,144436500.0,346976750.0,679683950.0,1.024398e+09
150,2000-08-07,53.562500,6924000,148.125000,4159800,0.866776,0.920835,0.800864,0.681607,0.507283,...,0.097569,21384700.0,42285100.0,56073100.0,78453500.0,101817500.0,147698000.0,346047650.0,677746250.0,1.019302e+09
151,2000-08-08,53.500000,3458500,148.687500,3658700,0.883284,0.959711,0.783472,0.781039,0.604694,...,0.124229,18929000.0,42092900.0,56834500.0,78638800.0,100634100.0,147961500.0,337284300.0,665569800.0,1.011038e+09
152,2000-08-09,52.312500,6305600,147.437500,5383800,0.896887,0.872593,0.836329,0.863721,0.665483,...,0.107975,16688100.0,37841700.0,58486100.0,80657300.0,102408300.0,149121000.0,336886700.0,659986250.0,1.008629e+09
153,2000-08-10,51.875000,4979200,146.718750,4193400,0.907191,0.994401,0.905623,0.895037,0.790239,...,0.085479,14743300.0,36128000.0,59909200.0,79519100.0,102799000.0,149201800.0,333940800.0,656889300.0,1.005238e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5969,2023-09-25,146.449997,7044700,432.230011,70874500,0.908433,0.545786,0.956229,0.517647,-0.109959,...,0.029168,22168800.0,43956600.0,90504700.0,129324400.0,165868800.0,245758900.0,551979000.0,945080600.0,1.740277e+09
5970,2023-09-26,144.929993,7468400,425.880005,96168400,0.916233,0.977418,0.962914,0.776421,0.081516,...,0.037255,21584800.0,44912600.0,89609900.0,128938100.0,168243100.0,248183000.0,545238900.0,939618100.0,1.737749e+09
5971,2023-09-27,145.779999,6717800,426.049988,104705800,0.924372,0.842778,0.917757,0.952805,0.169413,...,0.050795,21230900.0,43687000.0,88001800.0,127026700.0,168623900.0,244600500.0,543935100.0,929246700.0,1.733359e+09
5972,2023-09-28,147.589996,8868100,428.519989,92258300,0.933348,0.966230,0.515078,0.906355,0.215634,...,0.055156,23054300.0,45223100.0,86835000.0,127521100.0,171214700.0,246490400.0,543868300.0,926855700.0,1.734159e+09


In [93]:
ticker.to_csv("AMZN_feature.csv")